In [30]:
# IMDB 영화 리뷰 분석
with open('C:\\Workspace\\python\\Data_Science\\dataA\\labeledTrainData.tsv', encoding = 'UTF-8')as f:
    file = f.read()

In [31]:
len(file)

33550296

In [32]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:\\Workspace\\python\\Data_Science\\dataA\\labeledTrainData.tsv', sep = '\t', quoting = 3)
df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [33]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

In [34]:
# <br /> 태그는 공백으로 전환
df['review'] = df.review.str.replace('<br />', ' ')

In [35]:
# 구둣점, 숫자 제거 --> 영어 이외의 문자는 공백으로 변환
import re
df['review'] = df.review.apply(lambda x : re.sub('[^A-Za-z]', ' ', x))

In [36]:
df.review[0][:1000]

' With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for 

In [37]:
df.sentiment.value_counts()

1    12500
0    12500
Name: sentiment, dtype: int64

In [38]:
# 학습 및 테스트 데이터 셋으로 분리

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, test_size = 0.2, # 훈련용 20000개, 테스트용 5000개
    stratify = df.sentiment,random_state = 2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

In [39]:
# 텍스트 변환 - CounterVectorizer, ngram_range 활용
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [40]:
#이렇게 변환하면 안됨.
cvect1 = CountVectorizer(stop_words = 'english')
X_train_cv1 = cvect1.fit_transform(X_train)
X_test_cv1 = cvect1.fit_transform(X_test)
X_train_cv1.shape, X_test_cv1.shape

((20000, 66971), (5000, 37187))

In [41]:
# train, test data set에 있는 동일한 단어에 대해서 동일한 값으로 변환하기 위해 반드시 아래와 같이 해야함
cvect1 = CountVectorizer(stop_words = 'english')
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)
X_train_cv1.shape, X_test_cv1.shape

((20000, 66971), (5000, 66971))

In [42]:
# 로지스틱 회귀모델 연습
from sklearn.linear_model import LogisticRegression

In [43]:
# 객체 생성 및 훈련 - 훈련 데이터의 X와 y 모두필요
lr1 = LogisticRegression(max_iter = 500)
%time lr1.fit(X_train_cv1, y_train)

Wall time: 3.9 s


LogisticRegression(max_iter=500)

In [44]:
# 테스트 데이터에 대해 예측
pred1 = lr1.predict(X_test_cv1)
pred1

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [45]:
cvect2 = CountVectorizer(stop_words = 'english', ngram_range = (1, 2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)
X_train_cv2.shape, X_test_cv2.shape

((20000, 1461350), (5000, 1461350))

In [46]:
lr2 = LogisticRegression(max_iter = 500)
%time lr2.fit(X_train_cv2, y_train)

Wall time: 31 s


LogisticRegression(max_iter=500)

In [47]:
pred2 = lr2.predict(X_test_cv2)
pred2

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [48]:
# TfidVectorizer, ngram_range = (1, 1)
tvect1 = TfidfVectorizer(stop_words = 'english')
tvect1.fit(X_train)
X_train_tv1 = tvect1.transform(X_train)
X_test_tv1 = tvect1.transform(X_test)
X_train_tv1.shape, X_test_tv1.shape

((20000, 66971), (5000, 66971))

In [49]:
lr3 = LogisticRegression(max_iter = 500)
%time lr3.fit(X_train_tv1, y_train)

Wall time: 1.11 s


LogisticRegression(max_iter=500)

In [50]:
pred3 = lr3.predict(X_test_tv1)
pred3

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [51]:
# TfidVectorizer, ngram_range = (1, 2)
tvect2 = TfidfVectorizer(stop_words = 'english', ngram_range = (1, 2))
tvect2.fit(X_train)
X_train_tv2 = tvect2.transform(X_train)
X_test_tv2 = tvect2.transform(X_test)
X_train_tv2.shape, X_test_tv2.shape

((20000, 1461350), (5000, 1461350))

In [52]:
lr4 = LogisticRegression(max_iter = 500)
%time lr4.fit(X_train_tv2, y_train)

Wall time: 9.45 s


LogisticRegression(max_iter=500)

In [53]:
pred4 = lr4.predict(X_test_tv2)
pred4

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [54]:
# 평가
from sklearn.metrics import accuracy_score

In [55]:
score1 = accuracy_score(y_test, pred1)
score2 = accuracy_score(y_test, pred2)
print(f'{score1} --> {score2}')

0.8832 --> 0.8914


In [56]:
score3 = accuracy_score(y_test, pred3)
score4 = accuracy_score(y_test, pred4)
print(f'{score3} --> {score4}')

0.893 --> 0.883


In [62]:
# 비지도 학습 기반 감성분석
# 감성어휘사전
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sanalyer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Bestc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [63]:
senti_score = sanalyer.polarity_scores(df.review[0])
senti_score

{'neg': 0.13, 'neu': 0.743, 'pos': 0.127, 'compound': -0.7943}

In [60]:
# compund score 값이 0.1 보다 크면 긍정(1), 낮으면 부정(0)
def get_sentiment(sanalyer, review, threshold):
    score = sanalyer.polarity_scores(review)
    compound_score = score['compound']
    final_score = 1 if compound_score >= threshold else 0
    return final_score

In [64]:
df['vader_pred'] = df.review.apply(lambda x: get_sentiment(sanalyer, x, 0.1))

In [69]:
# 평가, real y - df.review.apply(lambda x: get_sentiment(sanalyzer, x, 0.1))
accuracy_score(df.sentiment, df.vader_pred)

0.69556

In [70]:
for threshold in [0, 0.1, 0.2, 0.3]:
    df['vader_pred'] = df.review.apply(lambda x: get_sentiment(sanalyer, x, threshold))
    score = accuracy_score(df.sentiment, df.vader_pred)
    print(f'threhold값 {threshold} : {score}')

threhold값 0 : 0.69104
threhold값 0.1 : 0.69556
threhold값 0.2 : 0.69952
threhold값 0.3 : 0.70264
